In [5]:
! pip install gym tensorflow

  Using cached gym-0.26.2.tar.gz (721 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached gym_notices-0.1.0-py3-none-any.whl.metadata (1.2 kB)
Using cached gym_notices-0.1.0-py3-none-any.whl (3.3 kB)
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827726 sha256=505f4ab131d29389f4bf416888f4c2d97501be60fd22758f15d67d5e14dacd44
  Stored in directory: /Users/imac/Library/Caches/pip/wheels/95/51/6c/9bb05ebbe7c5cb8171dfaa3611f32622ca4658d53f31c79077
Successfully built gym
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [gym]2/3 [gym]


In [6]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

env = gym.make("CartPole-v1")
obs_space = env.observation_space.shape[0]
act_space = env.action_space.n


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [7]:
gamma=0.99
learning_rate=0.01
num_episodes=1000
batch_size=32